# TOV Star Evolution

This notebook demonstrates the numerical evolution of a Tolman-Oppenheimer-Volkoff (TOV) neutron star using the BSSN formalism with relativistic hydrodynamics.

## Physical Setup

A TOV star is a spherically symmetric, static solution to Einstein's equations coupled with a perfect fluid. The equilibrium configuration satisfies:

**TOV Equations:**
$$\frac{dP}{dr} = -\frac{(\rho + P)(m + 4\pi r^3 P)}{r(r - 2m)}$$

$$\frac{dm}{dr} = 4\pi r^2 \rho$$

**Equation of State (Polytrope):**
$$P = K \rho_0^\Gamma$$

where $K$ is the polytropic constant, $\Gamma$ is the adiabatic index, and $\rho_0$ is the rest-mass density.

**Isotropic Coordinates:**

We work in isotropic coordinates where the spatial metric is conformally flat:
$$ds^2 = -\alpha^2 dt^2 + \psi^4 (dr^2 + r^2 d\Omega^2)$$

with $\psi = e^\phi$ the conformal factor.

---
## 1. Setup and Imports

In [ ]:
# Restart kernel to clear any previous state
# %reset -f

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import simpson
from tqdm.notebook import tqdm
import sys
import os

%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 12

# Add engrenage to path
sys.path.insert(0, '../..')

# Core modules
from source.core.grid import Grid
from source.core.spacing import LinearSpacing, NUM_GHOSTS
from source.core.statevector import StateVector
from source.backgrounds.sphericalbackground import FlatSphericalBackground

# BSSN
from source.bssn.bssnvars import BSSNVars
from source.bssn.bssnstatevariables import NUM_BSSN_VARS

# Hydro
from source.matter.hydro.perfect_fluid import PerfectFluid
from source.matter.hydro.eos import IdealGasEOS
from source.matter.hydro.reconstruction import create_reconstruction
from source.matter.hydro.riemann import HLLRiemannSolver
from source.matter.hydro.atmosphere import AtmosphereParams

# TOV modules
from examples.TOV.tov_solver import load_or_solve_tov_iso
import examples.TOV.tov_initial_data_interpolated as tov_id
import examples.TOV.utils_TOVEvolution as utils
from examples.TOV.utils_TOVEvolution import evolve_fixed_timestep, compute_baryon_mass

print("Imports successful!")

---
## 2. Star Configuration

Configure the TOV star parameters. The reference model from Font et al. (2002) uses:
- $K = 100$, $\Gamma = 2$
- $\rho_c = 1.28 \times 10^{-3}$ (central density)
- Compactness $M/R \approx 0.15$

In [ ]:
# =============================================================================
# STAR PARAMETERS (edit these!)
# =============================================================================
K = 100.0              # Polytropic constant
Gamma = 2.0            # Adiabatic index
rho_central = 1.28e-3  # Central density

# =============================================================================
# GRID PARAMETERS
# =============================================================================
r_max = 16.0           # Outer boundary (should be > 2*R_star)
num_points = 100       # Number of grid points

# =============================================================================
# EVOLUTION PARAMETERS
# =============================================================================
t_final = 2000.0       # Final evolution time (in code units ~ M_sun)
EVOLUTION_MODE = "cowling"  # "cowling" (fixed spacetime) or "dynamic" (full GR)
CFL = 0.1              # CFL factor for timestep

print(f"Star Configuration:")
print(f"  K = {K}, Gamma = {Gamma}")
print(f"  rho_central = {rho_central:.2e}")
print(f"\nGrid: N = {num_points}, r_max = {r_max}")
print(f"Evolution: t_final = {t_final}, mode = {EVOLUTION_MODE}")

---
## 3. Solve TOV Equations

Integrate the TOV equations to obtain the equilibrium stellar structure.

In [ ]:
# Solve TOV equations
print("Solving TOV equations...")
tov_solution = load_or_solve_tov_iso(
    K=K, Gamma=Gamma, rho_central=rho_central,
    r_max=r_max, accuracy="high"
)

print(f"\n" + "="*50)
print(f"TOV STAR PROPERTIES")
print(f"="*50)
print(f"  Gravitational mass: M = {tov_solution.M_star:.6f} M_sun")
print(f"  Isotropic radius:   R_iso = {tov_solution.R_iso:.3f} M_sun")
print(f"  Schwarzschild radius: R_schw = {tov_solution.R_schw:.3f} M_sun")
print(f"  Compactness:        C = M/R = {tov_solution.C:.4f}")
print(f"="*50)

In [ ]:
# Plot TOV solution
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

r_tov = tov_solution.r
R_star = tov_solution.R_iso

# Density
axes[0, 0].semilogy(r_tov, tov_solution.rho_baryon, 'b-', lw=2)
axes[0, 0].axvline(R_star, color='gray', ls=':', label=f'R = {R_star:.2f}')
axes[0, 0].set_xlabel('r')
axes[0, 0].set_ylabel(r'$\rho_0$')
axes[0, 0].set_title('Rest-mass Density')
axes[0, 0].legend()
axes[0, 0].set_xlim(0, r_max)

# Pressure
axes[0, 1].semilogy(r_tov, tov_solution.P + 1e-20, 'g-', lw=2)
axes[0, 1].axvline(R_star, color='gray', ls=':')
axes[0, 1].set_xlabel('r')
axes[0, 1].set_ylabel('P')
axes[0, 1].set_title('Pressure')
axes[0, 1].set_xlim(0, r_max)

# Enclosed mass
axes[0, 2].plot(r_tov, tov_solution.M, 'r-', lw=2)
axes[0, 2].axvline(R_star, color='gray', ls=':')
axes[0, 2].axhline(tov_solution.M_star, color='k', ls='--', alpha=0.5, label=f'M = {tov_solution.M_star:.3f}')
axes[0, 2].set_xlabel('r')
axes[0, 2].set_ylabel('M(r)')
axes[0, 2].set_title('Enclosed Mass')
axes[0, 2].legend()
axes[0, 2].set_xlim(0, r_max)

# Lapse
axes[1, 0].plot(r_tov, tov_solution.alpha, 'purple', lw=2)
axes[1, 0].axvline(R_star, color='gray', ls=':')
axes[1, 0].set_xlabel('r')
axes[1, 0].set_ylabel(r'$\alpha$')
axes[1, 0].set_title('Lapse Function')
axes[1, 0].set_xlim(0, r_max)

# Conformal factor
phi = 0.25 * np.log(tov_solution.exp4phi)
axes[1, 1].plot(r_tov, phi, 'teal', lw=2)
axes[1, 1].axvline(R_star, color='gray', ls=':')
axes[1, 1].set_xlabel('r')
axes[1, 1].set_ylabel(r'$\phi$')
axes[1, 1].set_title('Conformal Factor')
axes[1, 1].set_xlim(0, r_max)

# Metric function psi^2
psi2 = np.sqrt(tov_solution.exp4phi)
axes[1, 2].plot(r_tov, psi2, 'orange', lw=2)
axes[1, 2].axvline(R_star, color='gray', ls=':')
axes[1, 2].set_xlabel('r')
axes[1, 2].set_ylabel(r'$\psi^2$')
axes[1, 2].set_title('Conformal Factor $\psi^2 = e^{2\phi}$')
axes[1, 2].set_xlim(0, r_max)

plt.suptitle('TOV Equilibrium Solution', fontsize=14)
plt.tight_layout()
plt.show()

---
## 4. Create Initial Data

Interpolate the TOV solution onto the evolution grid and set up BSSN + hydro variables.

In [ ]:
# Setup grid and matter
spacing = LinearSpacing(num_points, r_max)
eos = IdealGasEOS(gamma=Gamma)

# Atmosphere configuration
atmosphere = AtmosphereParams(
    rho_floor=1.0e-10,
    p_floor=K * (1.0e-10)**Gamma,
    v_max=0.999,
    W_max=100.0,
    conservative_floor_safety=0.999
)

# Hydro setup
hydro = PerfectFluid(
    eos=eos,
    spacetime_mode="dynamic",
    atmosphere=atmosphere,
    reconstructor=create_reconstruction("wenoz"),
    riemann_solver=HLLRiemannSolver(atmosphere=atmosphere)
)

state_vector = StateVector(hydro)
grid = Grid(spacing, state_vector)
background = FlatSphericalBackground(grid.r)
hydro.background = background

print(f"Grid created: N = {grid.N}, dr_min = {grid.min_dr:.4f}")
print(f"Atmosphere: rho_floor = {atmosphere.rho_floor:.2e}")

In [ ]:
# Create initial data from TOV solution
print("Creating initial data...")
initial_state, prim_tuple = tov_id.create_initial_data_iso(
    tov_solution, grid, background, eos,
    atmosphere=atmosphere,
    polytrope_K=K, polytrope_Gamma=Gamma,
    interp_order=11
)

rho0_init, vr_init, p_init, eps_init = prim_tuple
print(f"Initial data created successfully!")
print(f"  Central density: rho_c = {rho0_init[NUM_GHOSTS]:.6e}")
print(f"  Max velocity: |v|_max = {np.max(np.abs(vr_init)):.2e}")

In [ ]:
# Plot initial data comparison
from scipy.interpolate import interp1d

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

r = grid.r
interior = slice(NUM_GHOSTS, -NUM_GHOSTS)

# Interpolate TOV to grid
rho_tov_interp = interp1d(tov_solution.r, tov_solution.rho_baryon, 
                          kind='cubic', fill_value=0, bounds_error=False)(r)
P_tov_interp = interp1d(tov_solution.r, tov_solution.P,
                        kind='cubic', fill_value=0, bounds_error=False)(r)

# Density comparison
axes[0].semilogy(r[interior], rho_tov_interp[interior] + 1e-20, 'k-', lw=2, label='TOV (analytic)')
axes[0].semilogy(r[interior], rho0_init[interior] + 1e-20, 'b--', lw=1.5, label='Initial data')
axes[0].axvline(R_star, color='gray', ls=':', alpha=0.7)
axes[0].set_xlabel('r')
axes[0].set_ylabel(r'$\rho_0$')
axes[0].set_title('Density')
axes[0].legend()

# Pressure comparison
axes[1].semilogy(r[interior], P_tov_interp[interior] + 1e-20, 'k-', lw=2, label='TOV')
axes[1].semilogy(r[interior], p_init[interior] + 1e-20, 'g--', lw=1.5, label='Initial data')
axes[1].axvline(R_star, color='gray', ls=':', alpha=0.7)
axes[1].set_xlabel('r')
axes[1].set_ylabel('P')
axes[1].set_title('Pressure')
axes[1].legend()

# Velocity (should be zero)
axes[2].plot(r[interior], vr_init[interior], 'r-', lw=1.5)
axes[2].axhline(0, color='k', ls='--', alpha=0.5)
axes[2].axvline(R_star, color='gray', ls=':', alpha=0.7)
axes[2].set_xlabel('r')
axes[2].set_ylabel(r'$v^r$')
axes[2].set_title('Radial Velocity (should be ~0)')

plt.suptitle('Initial Data vs TOV Solution', fontsize=14)
plt.tight_layout()
plt.show()

---
## 5. Time Evolution

Evolve the TOV star using RK4 integration. In **Cowling approximation**, the spacetime (BSSN variables) remains frozen and only the hydrodynamics evolves.

In [ ]:
# Import evolution functions
from examples.TOV.TOVEvolution import rk4_step, rk4_step_dynamic, _apply_atmosphere_reset

# Setup BSSN fixed data for Cowling
bssn_fixed = initial_state[:NUM_BSSN_VARS, :].copy()
bssn_d1_fixed = grid.get_d1_metric_quantities(initial_state)

# Select integrator based on mode
if EVOLUTION_MODE == "cowling":
    rk4_step_func = rk4_step
    print("Using COWLING approximation (fixed spacetime)")
else:
    def rk4_step_wrapper(state_flat, dt, grid, background, hydro,
                        bssn_fixed_unused, bssn_d1_fixed_unused, atmosphere):
        return rk4_step_dynamic(state_flat, dt, grid, background, hydro, atmosphere)
    rk4_step_func = rk4_step_wrapper
    print("Using DYNAMIC evolution (full BSSN + hydro)")

# Compute timestep
dt = CFL * grid.min_dr
num_steps = int(t_final / dt)
print(f"\nTimestep: dt = {dt:.6f}")
print(f"Total steps: {num_steps}")

In [ ]:
# Evolution with progress tracking
print(f"\nEvolving to t = {t_final}...")

# Storage for diagnostics
times = [0.0]
rho_central_series = [rho0_init[NUM_GHOSTS]]
Mb_series = []

# Compute initial baryon mass
bssn_vars = BSSNVars(grid.N)
bssn_vars.set_bssn_vars(initial_state[:NUM_BSSN_VARS, :])
hydro.set_matter_vars(initial_state, bssn_vars, grid)
rho0, vr, p, eps, W, h, _ = hydro._get_primitives(bssn_vars, grid.r)
Mb0 = compute_baryon_mass(grid, initial_state, rho0, vr, p, eps, W, h)
Mb_series.append(Mb0)

# Checkpoints for plotting
checkpoint_steps = [num_steps // 3, 2 * num_steps // 3, num_steps]
checkpoint_states = {0: initial_state.copy()}
checkpoint_times = {0: 0.0}

# Evolution loop
state_flat = initial_state.flatten()
save_interval = max(1, num_steps // 500)  # Save ~500 points for time series

with tqdm(total=num_steps, desc="Evolution", unit="step") as pbar:
    for step in range(num_steps):
        # RK4 step
        state_flat = rk4_step_func(state_flat, dt, grid, background, hydro,
                                   bssn_fixed, bssn_d1_fixed, atmosphere)
        
        t_curr = (step + 1) * dt
        
        # Save time series data
        if (step + 1) % save_interval == 0:
            s2d = state_flat.reshape((grid.NUM_VARS, grid.N))
            bssn_vars.set_bssn_vars(s2d[:NUM_BSSN_VARS, :])
            hydro.set_matter_vars(s2d, bssn_vars, grid)
            rho0, vr, p, eps, W, h, _ = hydro._get_primitives(bssn_vars, grid.r)
            
            times.append(t_curr)
            rho_central_series.append(rho0[NUM_GHOSTS])
            Mb_series.append(compute_baryon_mass(grid, s2d, rho0, vr, p, eps, W, h))
        
        # Save checkpoints
        if (step + 1) in checkpoint_steps:
            cp_idx = checkpoint_steps.index(step + 1) + 1
            checkpoint_states[cp_idx] = state_flat.reshape((grid.NUM_VARS, grid.N)).copy()
            checkpoint_times[cp_idx] = t_curr
            pbar.set_postfix({"checkpoint": cp_idx})
        
        pbar.update(1)

# Convert to arrays
times = np.array(times)
rho_central_series = np.array(rho_central_series)
Mb_series = np.array(Mb_series)

print(f"\nEvolution complete!")
print(f"Final time: t = {checkpoint_times[3]:.2f}")

---
## 6. Evolution Diagnostics

Analyze the evolution results: density profiles, velocity, errors, and conservation.

In [ ]:
# Extract primitives at checkpoints
def get_primitives(state):
    bssn = BSSNVars(grid.N)
    bssn.set_bssn_vars(state[:NUM_BSSN_VARS, :])
    hydro.set_matter_vars(state, bssn, grid)
    return hydro._get_primitives(bssn, grid.r)

prims = {}
for cp, state in checkpoint_states.items():
    rho0, vr, p, eps, W, h, success = get_primitives(state)
    prims[cp] = {'rho': rho0, 'vr': vr, 'p': p, 'W': W}

print("Primitives extracted for all checkpoints.")

In [ ]:
# Combined diagnostics plot (3x2)
fig, axes = plt.subplots(3, 2, figsize=(14, 12))
interior = slice(NUM_GHOSTS, -NUM_GHOSTS)
colors = plt.cm.viridis(np.linspace(0, 0.9, 4))

# Row 1: Density evolution
ax = axes[0, 0]
for i, (cp, t) in enumerate(checkpoint_times.items()):
    ax.semilogy(r[interior], prims[cp]['rho'][interior], color=colors[i], 
                lw=1.5, label=f't = {t:.0f}')
ax.axvline(R_star, color='gray', ls=':', alpha=0.7)
ax.set_xlabel('r')
ax.set_ylabel(r'$\rho_0$')
ax.set_title('Density Evolution')
ax.legend()

# Row 1: Pressure evolution
ax = axes[0, 1]
for i, (cp, t) in enumerate(checkpoint_times.items()):
    ax.semilogy(r[interior], prims[cp]['p'][interior] + 1e-20, color=colors[i], 
                lw=1.5, label=f't = {t:.0f}')
ax.axvline(R_star, color='gray', ls=':', alpha=0.7)
ax.set_xlabel('r')
ax.set_ylabel('P')
ax.set_title('Pressure Evolution')
ax.legend()

# Row 2: Velocity evolution
ax = axes[1, 0]
for i, (cp, t) in enumerate(checkpoint_times.items()):
    ax.plot(r[interior], prims[cp]['vr'][interior], color=colors[i], 
            lw=1.5, label=f't = {t:.0f}')
ax.axvline(R_star, color='gray', ls=':', alpha=0.7)
ax.axhline(0, color='k', ls='--', alpha=0.3)
ax.set_xlabel('r')
ax.set_ylabel(r'$v^r$')
ax.set_title('Radial Velocity')
ax.legend()

# Row 2: Density error
ax = axes[1, 1]
rho_ref = prims[0]['rho']
for i, (cp, t) in enumerate(checkpoint_times.items()):
    if cp == 0:
        continue
    delta_rho = np.abs(prims[cp]['rho'][interior] - rho_ref[interior])
    rel_err = delta_rho / (np.abs(rho_ref[interior]) + 1e-20)
    ax.semilogy(r[interior], rel_err + 1e-20, color=colors[i], lw=1.5, label=f't = {t:.0f}')
ax.axvline(R_star, color='gray', ls=':', alpha=0.7)
ax.set_xlabel('r')
ax.set_ylabel(r'$|\Delta\rho|/\rho_0$')
ax.set_title('Relative Density Error')
ax.legend()

# Row 3: Baryon mass conservation
ax = axes[2, 0]
delta_Mb = np.abs(Mb_series - Mb_series[0])
ax.semilogy(times, delta_Mb + 1e-20, 'b-', lw=1.5)
ax.set_xlabel('t')
ax.set_ylabel(r'$|M_b - M_{b,0}|$')
ax.set_title('Baryon Mass Deviation')

# Row 3: Central density oscillation
ax = axes[2, 1]
rho_c0 = rho_central_series[0]
delta_rho_c = (rho_central_series - rho_c0) / rho_c0
ax.plot(times, delta_rho_c, 'r-', lw=0.8)
ax.set_xlabel('t')
ax.set_ylabel(r'$(\rho_c - \rho_{c,0})/\rho_{c,0}$')
ax.set_title('Central Density Oscillation')

plt.suptitle(f'TOV Evolution Diagnostics ({EVOLUTION_MODE.upper()} mode)', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Print summary statistics
print("="*60)
print("EVOLUTION SUMMARY")
print("="*60)
print(f"\nCentral density:")
print(f"  Initial: {rho_c0:.6e}")
print(f"  Final:   {rho_central_series[-1]:.6e}")
print(f"  Max oscillation: {np.max(np.abs(delta_rho_c))*100:.3f}%")

print(f"\nBaryon mass conservation:")
print(f"  Initial: {Mb_series[0]:.6f}")
print(f"  Final:   {Mb_series[-1]:.6f}")
print(f"  Max deviation: {np.max(delta_Mb):.2e}")

print(f"\nVelocity:")
print(f"  Max |v^r| at final: {np.max(np.abs(prims[3]['vr'])):.3e}")
print("="*60)

---
## 7. Quasi-Normal Mode Analysis

The oscillations of the central density contain information about the star's quasi-normal modes (QNMs). We compute the power spectrum and compare with theoretical predictions from Font et al. (2002).

**Theoretical QNM frequencies (Cowling approximation):**
| Mode | Frequency [kHz] |
|------|----------------|
| F    | 2.696          |
| H1   | 4.534          |
| H2   | 6.346          |
| H3   | 8.161          |

In [ ]:
# Import QNM analysis functions
from examples.TOV.plot_qnm_analysis import (
    compute_power_spectrum, detect_qnm_frequencies, print_frequency_table,
    FREQUENCIES_COWLING_KHZ, FREQUENCIES_FULL_GR_KHZ, FREQ_CONVERSION, M_SUN_SECONDS
)

# Select theoretical frequencies based on mode
if EVOLUTION_MODE == "cowling":
    theoretical_freqs = FREQUENCIES_COWLING_KHZ
    approx_name = "Cowling"
else:
    theoretical_freqs = FREQUENCIES_FULL_GR_KHZ
    approx_name = "Full GR"

print(f"Using {approx_name} approximation frequencies")
print(f"Frequency conversion: 1/M_sun = {FREQ_CONVERSION:.2f} kHz")

In [ ]:
# Compute power spectrum of central density oscillation
t_start = 100  # Skip initial transient
freq_khz, power = compute_power_spectrum(times, delta_rho_c, t_start=t_start, window='hann')

# Detect peaks near theoretical frequencies
detected = detect_qnm_frequencies(freq_khz, power, theoretical_freqs, tolerance=0.5)

# Print results table
print_frequency_table(detected, title=f"QNM Analysis ({approx_name} approximation)")

In [ ]:
# QNM analysis plot
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Central density evolution
ax1 = axes[0]
ax1.plot(times, delta_rho_c, 'r-', lw=0.8)
ax1.set_xlabel('t [M]', fontsize=12)
ax1.set_ylabel(r'$(\rho_c - \rho_{c,0})/\rho_{c,0}$', fontsize=12)
ax1.set_title('Central Density Relative Change', fontsize=14)
ax1.grid(True, alpha=0.3)

# Inset: zoom on early oscillations
ax_inset = inset_axes(ax1, width="45%", height="40%", loc='upper right',
                      bbox_to_anchor=(0.0, 0.0, 0.95, 0.95),
                      bbox_transform=ax1.transAxes)
mask_zoom = times <= min(1000, t_final/4)
ax_inset.plot(times[mask_zoom], rho_central_series[mask_zoom]/rho_c0, 'k-', lw=0.6)
ax_inset.set_xlabel('t', fontsize=9)
ax_inset.set_ylabel(r'$\rho_c/\rho_{c,0}$', fontsize=9)
ax_inset.tick_params(labelsize=8)

# Right: Power spectrum
ax2 = axes[1]
max_freq = 14.0
ax2.semilogy(freq_khz, power, color='#8B0000', lw=1.5, label='Simulation')

# Mark theoretical frequencies
for mode, data in detected.items():
    f_theo = data['theoretical']
    f_det = data['detected']
    p_det = data['power']
    
    if f_theo < max_freq:
        ax2.axvline(f_theo, color='gray', ls='--', lw=1, alpha=0.7)
        ymax = ax2.get_ylim()[1] if ax2.get_ylim()[1] > 0 else np.max(power) * 2
        ax2.text(f_theo, ymax * 0.3, mode, ha='center', va='bottom',
                fontsize=11, fontweight='bold')
        
        if f_det is not None and p_det is not None:
            ax2.plot(f_det, p_det, 'bo', markersize=6, alpha=0.7)

ax2.set_xlabel('Frequency [kHz]', fontsize=12)
ax2.set_ylabel('Power', fontsize=12)
ax2.set_xlim(0, max_freq)
ax2.set_title(f'Power Spectrum - QNM Frequencies ({approx_name})', fontsize=14)
ax2.grid(True, which='major', ls=':', alpha=0.3)

plt.tight_layout()
plt.show()

---
## 8. Summary

This notebook demonstrated:

1. **TOV Solution**: Solving the TOV equations for a polytropic neutron star in isotropic coordinates
2. **Initial Data**: Creating BSSN + hydro initial data from the TOV solution
3. **Evolution**: Time integration using RK4 with HRSC methods for hydrodynamics
4. **Diagnostics**: Monitoring conservation laws and error growth
5. **QNM Analysis**: Extracting quasi-normal mode frequencies from central density oscillations

### Next Steps

- Try different star configurations (change `rho_central`, `K`, `Gamma`)
- Compare Cowling vs full GR evolution (change `EVOLUTION_MODE`)
- Increase resolution (`num_points`) for convergence studies
- Extend evolution time for better frequency resolution